In [1]:
import functools
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
recommendee = "Fro116"
neighborhood_size = 8192
confidence_interval = 0.95
normalize_variance = False

In [3]:
anime = pd.read_csv("AnimeList.csv")
anime = anime[["anime_id", "title", "type"]]

In [4]:
df = pd.read_csv("UserAnimeList.csv")

In [5]:
len(df["username"].unique()), len(df["anime_id"].unique())

(283045, 14478)

In [6]:
filtered_df = df[["username", "anime_id", "my_score"]].loc[lambda x: x["my_score"] != 0]

In [7]:
def read_xml(file, username):
    import xml.etree.ElementTree as ET

    xml_data = open(file, "r").read()  # Read file
    root = ET.XML(xml_data)  # Parse XML

    data = []
    cols = []
    for i, child in enumerate(root):
        data.append([subchild.text for subchild in child])
        cols.append(child.tag)
    new_list = pd.DataFrame(data).T
    new_list.columns = cols

    df = new_list.loc[[0, 9]].T.dropna().rename({0: "anime_id", 9: "my_score"}, axis=1)
    df["username"] = username
    df["anime_id"] = df["anime_id"].astype(int)
    df["my_score"] = df["my_score"].astype(int)
    df["username"] = df["username"].astype(str)
    df = df.loc[lambda x: x["my_score"] != 0]
    df = df.reset_index(drop=True)
    return df


def add_user(full_df, xml_file, username):
    user_df = read_xml(xml_file, username)
    without_user = full_df.loc[lambda x: x["username"] != username]
    return pd.concat([without_user, user_df], ignore_index=True)

In [8]:
filtered_df = add_user(filtered_df, "user_profiles/Fro116.xml", "Fro116")

In [9]:
average_rating = filtered_df["my_score"].mean()
user_bias = (
    pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
        {"my_score": "user_bias"}, axis=1
    )
    - average_rating
)
anime_bias = (
    pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
        {"my_score": "anime_bias"}, axis=1
    )
    - average_rating
)

In [10]:
filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
    user_bias, on=["username"]
)
filtered_df["normalized_score"] = (
    filtered_df["my_score"]
    - filtered_df["anime_bias"]
    - filtered_df["user_bias"]
    - average_rating
)
filtered_df = filtered_df.set_index("username")
filtered_df = filtered_df.dropna()

In [11]:
filtered_df

,anime_id,my_score,anime_bias,user_bias,normalized_score
username,,,,,
karthiga,21,9,0.960564,-0.059898,0.605473
karthiga,59,7,0.040203,-0.059898,-0.474166
karthiga,74,7,0.316283,-0.059898,-0.750245
karthiga,120,7,0.309858,-0.059898,-0.743821
karthiga,178,7,-0.227338,-0.059898,-0.206624
...,...,...,...,...,...
temptemptemp,10040,6,-1.636717,-1.493860,1.636717
cinnamoroller,12963,10,-0.798860,2.506140,0.798860
inactiveX,5143,7,-0.652951,-0.493860,0.652951


In [12]:
if normalize_variance:
    # Should we normalize by variance?
    filtered_df.groupby("username")["normalized_score"].std().hist(bins=100)
    user_stds = (
        filtered_df.groupby("username")[["normalized_score"]]
        .std()
        .rename({"normalized_score": "user_std"}, axis=1)
    )
    user_stds = user_stds.dropna().loc[lambda x: x["user_std"] != 0]
    filtered_df = filtered_df.merge(user_stds, on="username")
    filtered_df["normalized_score"] /= filtered_df["user_std"]
    filtered_df = filtered_df.drop("user_std", axis=1)

In [13]:
user_subset = filtered_df.loc[[recommendee]].merge(
    filtered_df.reset_index(), on="anime_id"
)

In [14]:
adj_cos_corr_numerator = user_subset.groupby("username").apply(
    lambda x: np.dot(x["normalized_score_x"], x["normalized_score_y"])
)
adj_cos_corr_denom = filtered_df.groupby("username").apply(
    lambda x: np.sqrt(np.dot(x["normalized_score"], x["normalized_score"]))
)
adj_cos_corr_denom *= adj_cos_corr_denom.loc[recommendee]
adj_cos_corrs = pd.DataFrame(
    (adj_cos_corr_numerator / adj_cos_corr_denom), columns=["corr"]
)
adj_cos_corrs = adj_cos_corrs.dropna()

In [136]:
corrs = adj_cos_corrs.copy()
corrs["similarity"] = corrs["corr"].abs()
corrs["size"] = user_subset.groupby("username").size()
corrs = corrs.drop(
    recommendee
)  # Technically not needed because its a noop for new series, but its useful for debugging

corrs = corrs.loc[lambda x: x["size"] > 2]
corrs["corr_var"] = (1 - corrs["corr"] * corrs["corr"]) / (
    corrs["size"] - 2
)  # assume variance is the same as pearson correlation variance. This is an overestimate TODO fix
# corrs['corr_var'] = corrs["corr"].abs() # assume variance is equal to mean

corrs = corrs.sort_values(by="similarity").dropna()[-neighborhood_size:]

In [137]:
corrs[["similarity", "corr_var"]].describe()

,similarity,corr_var
count,8192.000000,8192.0
mean,0.083613,0.0
std,0.015314,0.0
min,0.067726,0.0
25%,0.072488,0.0
50%,0.079044,0.0
75%,0.090059,0.0
max,0.193013,0.0


In [138]:
score = (filtered_df.merge(pd.DataFrame(corrs), on="username")).dropna()

In [139]:
deltas = score.groupby("anime_id").apply(
    lambda x: np.dot(x["normalized_score"], x["corr"]) / x["corr"].abs().sum()
)
weights = score.groupby("anime_id").apply(lambda x: x["corr"].abs().sum())
counts = score.groupby("anime_id").size()

In [140]:
# add standard error of the weighted mean
user_var = (
    pd.DataFrame(filtered_df.groupby("username")["normalized_score"].var())
    .rename({"normalized_score": "user_var"}, axis=1)
    .dropna()
)
score = score.merge(user_var, on="username")

anime_var = (
    pd.DataFrame(filtered_df.groupby("anime_id")["normalized_score"].var())
    .rename({"normalized_score": "anime_var"}, axis=1)
    .dropna()
)
score = score.merge(anime_var, on="anime_id")


# This is the formula for the standard deviation of the delta. Delta
# is a weighted sum of the form δ = Σ(s_i * w_i) / (Σw_i), where s_i is
# a vector scores for user i and w_i is the weight for user_i.
#
# By linearity, it suffices to compute (s_i * w_i) / (Σw_i). We can compute
# Var(s_i) directly, by taking the variance over the vector s_i (i.e. over
# all items s_i has rated). We treat w_i as a random variable with mean w_i
# and variance corr['corr_var']
#
# The variance for (w_i) / (Σw_i) can be estimated by doing a Taylor Approximation.
# See equation 20 of https://www.stat.cmu.edu/~hseltman/files/ratio.pdf. The
# formula for the ratio of two correlated variables R,S is
# Var(R/S) = E[R]^2/E[S]^2(Var[R]/E[R]^2 - 2Cov(R,S)/(E[R]E[S]) + Var[S]/E[S]^2)
#
# Lastly we take the product distribution of s_i and (w_i) / (Σw_i).
def correction_factor(x):
    return (
        1
        + x["corr_var"] / (x["corr"] * x["corr"])
        - 2 * x["corr_var"] / (x["corr"].abs().sum() * x["corr"].abs())
        + x["corr_var"].sum() / (x["corr"].abs().sum() * x["corr"].abs().sum())
    )


# def poisson_correction_factor(x):
#     return (
#         1
#         + 1 / x["corr"].abs()
#         - 1 / (x["corr"].abs().sum())
#     )

# def wrong_correction_factor(x):
#     # emprically gives good results
#     return (
#         1 / x["corr"].abs()
#     )

# def exponential_correction_factor(x):
#     return (
#         1
#         + 1
#         - 2 * x["corr"].abs() / (x["corr"].abs().sum() - x["corr"].abs())
#         + (x["corr"] * x["corr"]).sum()
#         / (x["corr"].abs().sum() * x["corr"].abs().sum())
#     )


# delta_sem = score.groupby("anime_id").apply(
#     lambda x: np.sum(
#         x["user_var"]
#         * x["corr"].abs()
#         * x["corr"].abs()
#         / (x["corr"].abs().sum() * x["corr"].abs().sum())
#         + x["user_var"]
#     )
# )

delta_pred_var = score.groupby("anime_id").apply(
    lambda x: np.sum(
        x["user_var"] * x["corr"].abs() * x["corr"].abs() * correction_factor(x)
    )
    / (x["corr"].abs().sum() * x["corr"].abs().sum())
)

# The above is a biased estimator of the variance. To unbias the estimator,
# we need to apply a Bessel-like correction. See the formula in
# (https://stats.stackexchange.com/questions/47325/bias-correction-in-weighted-variance)
delta_pred_var = delta_pred_var.loc[counts > 1]
bias_correction = score.groupby("anime_id").apply(
    lambda x: (x["corr"].abs().sum() * x["corr"].abs().sum())
    / (x["corr"].abs().sum() * x["corr"].abs().sum() - (x["corr"] * x["corr"]).sum())
)
delta_pred_var *= bias_correction

# if the sem < 0, then the ratio distribution approximation failed,
# usually because sample size is too small
delta_pred_var.loc[lambda x: x < 0] = np.inf

# delta_pred_var is the variance of what the model computes delta to be
# there is still the error between what the model computes delta to be
# and the ground truth.
# delta_model_var = 0.5 / (weights - 1) # Formula for sample variance
# delta_model_var.loc[lambda x: x < 0] = np.inf
delta_model_var = 0

<ipython-input-140-0ea7b3cc2c14>:86: RuntimeWarning: divide by zero encountered in double_scalars
  lambda x: (x["corr"].abs().sum() * x["corr"].abs().sum())


In [141]:
delta_pred_var.sort_values()

anime_id
1535     0.000338
16498    0.000388
6547     0.000395
11757    0.000395
1575     0.000410
           ...   
3633     4.794900
32765    4.850491
1885     5.281169
12499    5.780140
3396     8.452906
Length: 8661, dtype: float64

In [143]:
pred_df = pd.DataFrame()
pred_df["delta"] = deltas
pred_df["weight"] = weights
pred_df["counts"] = counts
pred_df["delta_pred_sem"] = np.sqrt(delta_pred_var)
pred_df["delta_model_sem"] = np.sqrt(delta_model_var)

pred_df["delta_sem"] = np.sqrt(delta_pred_var + delta_model_var)
pred_df["blp"] = anime_bias + user_bias.loc[recommendee].squeeze() + average_rating
pred_df = pred_df.dropna()

recomendee_seen_shows = filtered_df.loc[recommendee].merge(pred_df, on=["anime_id"])
recomendee_seen_shows["target"] = (
    recomendee_seen_shows["my_score"] - recomendee_seen_shows["blp"]
)
model = lm("target ~ delta + 0", recomendee_seen_shows)
pred_df["score"] = model.predict(pred_df) + pred_df["blp"]
pred_df["sem"] = np.sqrt(
    (
        (
            pred_df["delta_sem"] * pred_df["delta_sem"]
            + pred_df["delta"] * pred_df["delta"]
        )
        * (
            model.bse["delta"] * model.bse["delta"]
            + model.params["delta"] * model.params["delta"]
        )
    )
    - pred_df["delta"]
    * pred_df["delta"]
    * model.params["delta"]
    * model.params["delta"]
)
zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)

pred_df["score_lower_bound"] = pred_df["score"] - pred_df["sem"] * zscore
pred_df["score_upper_bound"] = pred_df["score"] + pred_df["sem"] * zscore

pred_df = pred_df.merge(anime, on="anime_id")
pred_df = pred_df.set_index("anime_id")

In [144]:
# confirm that setting blp = 1 is reasonable
print(lm("my_score ~ delta + blp + 0", recomendee_seen_shows).summary())

                                 OLS Regression Results                                
Dep. Variable:               my_score   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          1.212e+04
Date:                Sun, 25 Apr 2021   Prob (F-statistic):                   3.29e-318
Time:                        02:32:27   Log-Likelihood:                         -407.59
No. Observations:                 344   AIC:                                      819.2
Df Residuals:                     342   BIC:                                      826.9
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [145]:
# confirm that the top shows are ones that the user rates highly
pred_df.sort_values(by="score_lower_bound")[-20:]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
396,0.689996,11.892178,144,0.110698,0.0,0.110698,6.813696,8.805875,0.325452,8.168001,9.443750,Seikai no Senki,TV
1689,0.454015,268.301257,3213,0.022883,0.0,0.022883,7.022911,8.333759,0.077197,8.182456,8.485062,Byousoku 5 Centimeter,Movie
962,0.464429,29.532838,353,0.070878,0.0,0.070878,7.262308,8.603225,0.208760,8.194063,9.012386,Aria The Natural,TV
29893,0.621297,9.231674,108,0.126768,0.0,0.126768,7.129651,8.923483,0.370222,8.197860,9.649105,Aria The Avvenire,Special
2759,0.521940,182.253897,2169,0.029085,0.0,0.029085,7.150698,8.657660,0.095708,8.470076,8.845244,Evangelion: 1.0 You Are (Not) Alone,Movie
12467,0.897850,54.732444,668,0.049556,0.0,0.049556,6.198381,8.790686,0.163432,8.470366,9.111006,Nazo no Kanojo X,TV
227,0.642466,206.466434,2481,0.027274,0.0,0.027274,6.835007,8.689958,0.096908,8.500021,8.879895,FLCL,OVA
820,0.354073,41.144860,497,0.064764,0.0,0.064764,7.872021,8.894313,0.189642,8.522622,9.266004,Ginga Eiyuu Densetsu,OVA
9756,0.433084,291.359410,3501,0.022871,0.0,0.022871,7.447999,8.698414,0.076233,8.548999,8.847828,Mahou Shoujo Madoka★Magica,TV


In [146]:
# Movies tend to be recaps of TV series
new_recs = pred_df.drop(filtered_df.loc[recommendee].anime_id, errors="ignore").loc[
    lambda x: (x["type"] != "Movie")
    & (x["type"] != "Special")
    & (x["type"] != "OVA")
    & (x["type"] != "ONA")
    & (x["type"] != "Music")
]

In [147]:
seen_shows = pred_df.loc[
    pred_df.index.intersection(filtered_df.loc[recommendee].anime_id)
]
seen_shows["my_score"] = filtered_df.loc[recommendee].set_index("anime_id")[
    ["my_score"]
]

In [148]:
new_recs.loc[lambda x: x["title"] == "Koi Kaze"]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
634,0.202164,24.066237,290,0.076947,0.0,0.076947,6.273674,6.857369,0.222975,6.420346,7.294392,Koi Kaze,TV


In [149]:
new_recs.loc[lambda x: x["title"] == "Pingu in the City"]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
36259,0.447833,7.131127,86,0.170714,0.0,0.170714,6.738497,8.031496,0.494685,7.061931,9.001061,Pingu in the City,TV


In [150]:
new_recs.loc[lambda x: x["title"] == "Casshern Sins"]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
4981,0.05141,31.411087,384,0.069762,0.0,0.069762,6.51621,6.664642,0.201564,6.269584,7.0597,Casshern Sins,TV


In [151]:
new_recs.loc[lambda x: x["title"] == "Mugen no Ryvius"]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
593,0.391514,9.408438,112,0.133788,0.0,0.133788,6.555974,7.686367,0.387984,6.925932,8.446802,Mugen no Ryvius,TV


In [152]:
new_recs.loc[lambda x: x["title"] == "Kodomo no Jikan (TV)"]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
2403,0.318898,31.566416,391,0.069206,0.0,0.069206,6.076891,6.997624,0.20186,6.601987,7.393262,Kodomo no Jikan (TV),TV


In [153]:
new_recs.loc[
    lambda x:  (x["delta"] > 0)
].sort_values(by="score_lower_bound")[-25:]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
1454,0.263653,17.838523,218,0.094507,0.0,0.094507,6.370114,7.131342,0.273970,6.594371,7.668313,Kemonozume,TV
1254,0.155547,20.713216,253,0.086299,0.0,0.086299,6.634840,7.083941,0.249655,6.594626,7.573256,Saint Seiya,TV
1088,0.060094,18.607045,228,0.090824,0.0,0.090824,6.935756,7.109261,0.262405,6.594956,7.623566,Macross,TV
85,0.100547,14.925202,185,0.106047,0.0,0.106047,6.911561,7.201865,0.306450,6.601233,7.802497,Mobile Suit Zeta Gundam,TV
2403,0.318898,31.566416,391,0.069206,0.0,0.069206,6.076891,6.997624,0.201860,6.601987,7.393262,Kodomo no Jikan (TV),TV
589,0.264206,4.993294,60,0.179414,0.0,0.179414,6.861303,7.624129,0.518770,6.607359,8.640899,Ginga Nagareboshi Gin,TV
1579,0.145740,14.878283,189,0.094145,0.0,0.094145,6.731344,7.152130,0.272246,6.618537,7.685723,Kiniro no Corda: Primo Passo,TV
1471,0.309874,2.530446,31,0.218052,0.0,0.218052,6.980432,7.875112,0.630446,6.639460,9.110764,City Hunter 2,TV
34973,0.277867,9.499969,115,0.122003,0.0,0.122003,6.543020,7.345287,0.353259,6.652913,8.037661,Love Live! Sunshine!! 2nd Season,TV


In [134]:
new_recs.loc[
    lambda x:  (x["delta"] > 0)
].sort_values(by="score_lower_bound")[-25:]

,delta,weight,counts,delta_pred_sem,delta_model_sem,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,,,
1254,0.155547,20.713216,253,0.204791,0.0,0.204791,6.634840,7.083941,0.591712,5.924206,8.243675,Saint Seiya,TV
18121,0.501206,11.190428,137,0.202691,0.0,0.202691,5.643353,7.090453,0.587140,5.939680,8.241226,Teekyuu 2,TV
147,0.075947,55.898825,674,0.105107,0.0,0.105107,6.362681,6.581957,0.303683,5.986750,7.177164,Kimi ga Nozomu Eien,TV
1491,0.564776,2.732970,33,0.412734,0.0,0.412734,6.696004,8.326643,1.193244,5.987927,10.665359,Ginga Tetsudou 999,TV
9996,0.598636,3.052283,37,0.400320,0.0,0.400320,6.545916,8.274320,1.157547,6.005570,10.543071,Hyouge Mono,TV
634,0.202164,24.066237,290,0.149739,0.0,0.149739,6.273674,6.857369,0.432897,6.008906,7.705832,Koi Kaze,TV
31771,0.255933,13.458854,168,0.174819,0.0,0.174819,6.278450,7.017388,0.505477,6.026672,8.008103,Amanchu!,TV
182,0.063375,35.077449,426,0.149270,0.0,0.149270,6.693356,6.876334,0.431212,6.031175,7.721494,Tenkuu no Escaflowne,TV
34973,0.277867,9.499969,115,0.231556,0.0,0.231556,6.543020,7.345287,0.669311,6.033461,8.657113,Love Live! Sunshine!! 2nd Season,TV
